In [16]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm  # Fortschrittsanzeige

# WebDriver mit WebDriver-Manager automatisch verwalten
service = Service(ChromeDriverManager().install())

# Chrome-Optionen für stabilen Betrieb
options = Options()
options.add_argument("--headless")  
options.add_argument("--no-sandbox")  
options.add_argument("--disable-dev-shm-usage")  
options.add_argument("--window-size=1920,1080")  

# WebDriver starten
driver = webdriver.Chrome(service=service, options=options)

# Ziel-URL
URL = "https://www.enforcementtracker.com/"
print("Lade die Webseite...")
driver.get(URL)
time.sleep(5)

# Warten, bis die Tabelle geladen ist
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))

# Fortschrittsanzeige einrichten
total_pages = 254  # Anzahl der Seiten
data = []

print("🔄 Starte Scraping über alle Seiten...\n")

for page in tqdm(range(1, total_pages + 1), desc="Scraping Fortschritt"):
    # HTML holen und mit BeautifulSoup verarbeiten
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table")

    # Daten extrahieren
    if table:
        rows = table.find_all("tr")[1:]  # Erste Zeile (Header) überspringen
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 5:
                data.append([col.text.strip() for col in cols[:5]])

    # Versuchen, zur nächsten Seite zu navigieren
    try:
        # Warte, bis der "Next"-Button sichtbar ist
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.ID, "penalties_next"))
        )

        # Klick erzwingen via JavaScript (um Blockierungen zu umgehen)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Warten, damit die neue Seite geladen wird

    except Exception as e:
        print(f"\n🚀 Letzte Seite erreicht oder Fehler: {e}")
        break

# WebDriver schließen
driver.quit()

# Daten speichern
df = pd.DataFrame(data, columns=["Country", "Date", "Company", "Amount", "Violation"])
csv_path = "enforcement_tracker_data.csv"
df.to_csv(csv_path, index=False, encoding="utf-8")

print(f"\n✅ Scraping abgeschlossen! {len(df)} Einträge gespeichert in '{csv_path}'")


Lade die Webseite...
🔄 Starte Scraping über alle Seiten...



Scraping Fortschritt: 100%|██████████| 254/254 [13:55<00:00,  3.29s/it]



✅ Scraping abgeschlossen! 2534 Einträge gespeichert in 'enforcement_tracker_data.csv'
